# Feast Prompt Templates

This example shows how to bind a LangChain prompt template to an existing Feast feature store
and have the prompt rendered with information from the (online) store.

_Note_: to run this code, you need to go first complete the **"Feast/Cassandra setup"** instructions given earlier.

## Initialization of the feature store

In [1]:
from feast import FeatureStore

feast_repo_path = "feast_store/user_features/feature_repo/"
store = FeatureStore(repo_path=feast_repo_path)

/home/stefano/.virtualenvs/langchain-cassio-3.10/lib/python3.10/site-packages/cassandra/io/asyncorereactor.py:34: DeprecationWarning: The asyncore module is deprecated and will be removed in Python 3.12. The recommended replacement is asyncio
  import asyncore


## Creation of the prompt template

In [2]:
from langchain.prompts import createFeastPromptTemplate

/home/stefano/personal/Datastax/code/my_github/langchain/langchain/vectorstores/analyticdb.py:20: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()  # type: Any


In [3]:
ecommercePrompt = """You are a helpful assistant for
customers of an e-commerce website.
You will give them a suggestion on what to do,
based on their profile and pattern of usage so far.
The customer information is summarized by the following values:

> Customer age:               {age}
> Number of purchases so far: {purchases}
> Site visits (per month):    {visit_frequency}
> Active cart now? (1 = yes): {active_cart}

USER QUESTION: {user_question}

YOUR SUGGESTION: """

In [4]:
ecommerceTemplate = createFeastPromptTemplate(
    store=store,
    template=ecommercePrompt,
    input_variables=["user_name", "user_question"],
    field_mapper={
        'age': ('user_data', 'age'),
        'purchases': ('user_data', 'purchases'),
        'visit_frequency': ('user_data', 'visit_frequency'),
        'active_cart': ('active_cart', 'active_cart'),
    },
)

## Formatting the template

### Full rendering

In [5]:
print(ecommerceTemplate.format(
    user_name='marilyn', user_question='How do I remove an item from the cart?'
))

You are a helpful assistant for
customers of an e-commerce website.
You will give them a suggestion on what to do,
based on their profile and pattern of usage so far.
The customer information is summarized by the following values:

> Customer age:               55
> Number of purchases so far: 12
> Site visits (per month):    0.17000000178813934
> Active cart now? (1 = yes): 0

USER QUESTION: How do I remove an item from the cart?

YOUR SUGGESTION: 


### Partialing the template

In [6]:
eCommPartial = ecommerceTemplate.partial(user_name='rodolfo')
print(eCommPartial.format(user_question='When is the next special offer?'))

You are a helpful assistant for
customers of an e-commerce website.
You will give them a suggestion on what to do,
based on their profile and pattern of usage so far.
The customer information is summarized by the following values:

> Customer age:               21
> Number of purchases so far: 323
> Site visits (per month):    8.59000015258789
> Active cart now? (1 = yes): 1

USER QUESTION: When is the next special offer?

YOUR SUGGESTION: 
